In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


Import some necessary library 

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

np.random.seed(42)


Pre-processing

In [3]:
#Load csv files from path
bad_data_path = '/content/gdrive/MyDrive/PPG_data/PPG signal/bad_data.csv'
bad_label_path = '/content/gdrive/MyDrive/PPG_data/PPG signal/bad_label.csv'
good_data_path = '/content/gdrive/MyDrive/PPG_data/PPG signal/good_data.csv'
good_label_path = '/content/gdrive/MyDrive/PPG_data/PPG signal/good_label.csv'


In [4]:
# Read dataframe from csv_files
bad_data = pd.read_csv(bad_data_path)
bad_label = pd.read_csv(bad_label_path)
good_data = pd.read_csv(good_data_path)
good_label = pd.read_csv(good_label_path)


FileNotFoundError: ignored

In [ ]:
bad_data_arr = np.array(bad_data)
good_data_arr = np.array(good_data)

bad_label_arr = np.array(bad_label)
good_label_arr = np.array(good_label)
print(bad_data_arr.shape)
print(good_data_arr.shape)
print(bad_label_arr.shape)
print(good_label_arr.shape)
data_arr = np.concatenate((bad_data_arr, good_data_arr))
label_arr = np.concatenate((bad_label_arr, good_label_arr))
print(data_arr.shape)
print(label_arr.shape)

In [ ]:
print(data_arr)

After doing some manipulations with the input data, we get 2 arrays:
The 'data_arr' include 1947 samples, each sample has 3000 features (has selected before) and the remaining array consists of 1947 elements (including only the number 0 and number 1) is the label of the above 1947 samplea.  

In [ ]:
print(data_arr.shape)

Create new-full-dataframe for easy processing!

In [ ]:
data = np.concatenate((data_arr,label_arr), axis = 1)
print(data)
print(data.shape)

In [ ]:
#Understanding the data
pd.DataFrame(data_arr).describe()

In [ ]:
df = pd.DataFrame(data)

In [ ]:
# print(df.shape)
# print(df.head())
print(df)

In [ ]:
#Visualize a sample
plt.style.use('fivethirtyeight')
df.iloc[0, :-1].plot(subplots=True,
                  fontsize=10, 
                  linewidth=0.5, 
                  title='Visualization of the transformed Features')
plt.show()

In [ ]:
#Shuffling the dataframe
from sklearn.utils import shuffle
df = shuffle(df)
print(df.head())

In [ ]:
print(type(df))

Divide the data X and label y

In [ ]:
X= df.iloc[:,:-1]
print(X)
X.shape

In [ ]:
y = df.iloc[:,-1].values
print(y)
y.shape

Scale the data X and categoricate y

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X)
x = scaler.transform(X)
from tensorflow.keras.utils import to_categorical
y = to_categorical(y)
print(y[300])

In [ ]:
print(x)

Split train data and test data with ratio is 1/5

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 4)

Reshape X_train, X_test to suitable for  input of model

In [ ]:
x_train = np.reshape(x_train, (x_train.shape[0],1,x.shape[1]))
x_test = np.reshape(x_test, (x_test.shape[0],1,x.shape[1]))
print(x_train.shape)
print(x_test.shape)

Build simple model to predict with 2 LSTM layers with inpput shape = (1,3000), activation of first LSTM layer is "relu" function and after that is "sigmoid"  function.
Because this is a binary classification, so we will use the Dense layer with hyperparameter is 2.
Loss function and optimizers as below.

In [ ]:
import tensorflow as tf
from tensorflow.keras import Sequential

from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import LSTM
tf.keras.backend.clear_session()

model = Sequential()
model.add(LSTM(64, input_shape=(1,3000),activation="relu",return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(32,activation="sigmoid"))
model.add(Dropout(0.2))
model.add(Dense(2, activation='sigmoid'))
from tensorflow.keras.optimizers import SGD
model.compile(loss = 'binary_crossentropy', optimizer = "adam", metrics = ['acc'])
model.summary()

Fiting splitted data
Number of epoch is 100 for the first running is 100 epochs, but after observering the learning curve chart, we choice 120 epochs 

In [ ]:
history = model.fit(x_train, y_train, epochs = 120, validation_data= (x_test, y_test))
score, acc = model.evaluate(x_test, y_test)

In [ ]:
from sklearn.metrics import accuracy_score
pred = model.predict(x_test)
predict_classes = np.argmax(pred,axis=1)
expected_classes = np.argmax(y_test,axis=1)
print(expected_classes.shape)
print(predict_classes.shape)
correct = accuracy_score(expected_classes,predict_classes)
print(f"Training Accuracy: {correct}")

Visualize the learning curve

In [ ]:
fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(18,6))
axs[0].plot(history.history['loss'], color='b', label='Training loss')
axs[0].plot(history.history['val_loss'], color='r', label='Validation loss')
axs[0].set_title("Loss curves")
# axs[0].legend(loc='best', shadow=True)
axs[0].legend()
axs[1].plot(history.history['acc'], color='b', label='Training accuracy')
axs[1].plot(history.history['val_acc'], color='r', label='Validation accuracy')
axs[1].set_title("Accuracy curves")
axs[1].legend(loc='best', shadow=True)
plt.show()

In [ ]:
model.save('my_model.h5')

# New Section